# Master's thesis - Lukas Meuris - graphCast training - loss functions

This notebook contains the code to train the graphCast model. 
We use this notebook to traint he graphcast model with different loss-functions to see the influence of it on the model performance.

 

In [ ]:
# @title Pip install graphcast and dependencies

#!pip install --upgrade https://github.com/deepmind/graphcast/archive/master.zip


# Installation and initialisation.


In [1]:
# @title Imports
import sys
sys.path.append("../")

import dataclasses
import functools

from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import data_utils
from graphcast import graphcast
from graphcast import rollout
from graphcast import normalization
from graphcast import xarray_jax
from graphcast import xarray_tree
import haiku as hk
import jax
import numpy as np
import xarray

import optax

import os
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Access data from GCS

In [2]:
# @title Authenticate with Google Cloud Storage

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")

# Load the Data and initialize the model

## Load the model parameters

We use random parameters for the model initialization. 
We'll get random predictions, but we can change the model architecture.


*Checkpoints vary across a few axes:*
- *The mesh size specifies the internal graph representation of the earth. Smaller meshes will run faster but will have worse outputs. The mesh size does not affect the number of parameters of the model.*
- *The resolution and number of pressure levels must match the data. Lower resolution and fewer levels will run a bit faster. Data resolution only affects the encoder/decoder.*
- *All our models predict precipitation. However, ERA5 includes precipitation, while HRES does not. Our models marked as "ERA5" take precipitation as input and expect ERA5 data as input, while model marked "ERA5-HRES" do not take precipitation as input and are specifically trained to take HRES-fc0 as input (see the data section below).*


In [3]:
# @title choose model parameters
random_mesh_size = 5 # mesh size: 4 - 6
random_gnn_msg_steps = 8 # message passing steps: 1 - 32
random_latent_size = 128 # latent size: 16,32,64,128,256,512
random_levels = 13 # levels: 13 or 37


In [4]:
# @title load the model parameters
params = None  # Filled in below
state = {}
model_config = graphcast.ModelConfig(
    resolution=0,
    mesh_size=random_mesh_size,
    latent_size=random_latent_size,
    gnn_msg_steps=random_gnn_msg_steps,
    hidden_layers=1,
    radius_query_fraction_edge_length=0.6)
task_config = graphcast.TaskConfig(
    input_variables=graphcast.TASK.input_variables,
    target_variables=graphcast.TASK.target_variables,
    forcing_variables=graphcast.TASK.forcing_variables,
    pressure_levels=graphcast.PRESSURE_LEVELS[random_levels],
    input_duration=graphcast.TASK.input_duration,
)
model_config

ModelConfig(resolution=0, mesh_size=5, latent_size=128, gnn_msg_steps=8, hidden_layers=1, radius_query_fraction_edge_length=0.6, mesh2grid_edge_normalization_factor=None)

# Load the ERA5 data


In [5]:
# @title Get and filter the list of available example datasets, then select one.

# Define the relative path to the file
relative_path = "ERA5_data/obs_data.zarr"

# Get the absolute path by joining the current directory with the relative path
absolute_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
print(absolute_path)

# Open the Zarr file using xarray
obs_data = xarray.open_zarr(absolute_path)

/home/jupyter-lukas/Masters-Thesis/ERA5_data/obs_data.zarr


In [ ]:
# view obs_data:
obs_data

## extract the training data

In [6]:
#time slice: 1980-01-01T00:00:00.000000000 to 2019-12-31T00:00:00.000000000 - TRAINING
train_data = obs_data.sel(time=slice('1980-01-01T00:00:00.000000000','2019-12-31T00:00:00.000000000'))

## extract the evaluation data

In [7]:
#time slice: 2020-01-01T00:00:00.000000000 to 2022-01-01T00:00:00.000000000 - EVALUATION
eval_data = obs_data.sel(time=slice('2020-01-01T00:00:00.000000000','2021-01-01T00:00:00.000000000'))

## choose number of training and evaluation steps.

In [8]:
# @title Choose training and eval data to extract
train_steps = 12  # {1 - obs_data.sizes["time"]-2} | 12 = 3days
eval_steps = 40 # {1 - obs_data.sizes["time"]-2} | 40 = 10days

## extract training and eval inputs, targets and forcings.

In [9]:

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    train_data, target_lead_times=slice("6h", f"{train_steps*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    eval_data, target_lead_times=slice("6h", f"{eval_steps*6}h"),
    **dataclasses.asdict(task_config))

## Load normalization data

In [10]:

with gcs_bucket.blob("stats/diffs_stddev_by_level.nc").open("rb") as f:
    diffs_stddev_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob("stats/mean_by_level.nc").open("rb") as f:
    mean_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob("stats/stddev_by_level.nc").open("rb") as f:
    stddev_by_level = xarray.load_dataset(f).compute()


## Build jitted functions, and possibly initialize random weights

In [11]:

def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor


@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)


@hk.transform_with_state
def loss_fn(model_config, task_config, inputs, targets, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  loss, diagnostics = predictor.loss(inputs, targets, forcings)
  return xarray_tree.map_structure(
      lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
      (loss, diagnostics))

def grads_fn(params, state, inputs, targets, forcings, model_config, task_config):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is required by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
  params, state = init_jitted(
      rng=jax.random.PRNGKey(0),
      inputs=train_inputs.compute(),
      targets_template=train_targets.compute(),
      forcings=train_forcings.compute())

loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = jax.jit(with_configs(grads_fn))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))

In [12]:
# setup optimiser
lr = 1e-3
optimiser = optax.adam(lr, b1=0.9, b2=0.999, eps=1e-8)
opt_state = optimiser.init(params)

# Model training loop

In [13]:
N = train_data.sizes['time'] - train_steps - 4
loss_array = []

for i in range(N):
    train_batch = train_data.isel(time=slice(i, i + train_steps + 2))
    train_batch = train_batch.compute()

    train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    train_batch, target_lead_times=slice("6h", f"{train_steps*6}h"),
    **dataclasses.asdict(task_config))

    # calculate loss and gradients
    loss, diagnostics, next_state, grads = grads_fn_jitted(params, state, train_inputs, train_targets, train_forcings)

    # update
    updates, opt_state = optimiser.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    
    loss_array.append(loss)
    if i%100 == 0:
        print("I:", i , " - Loss:", loss)



I: 0  - Loss: 29.640625
I: 100  - Loss: 7.328125
I: 200  - Loss: 6.7109375
I: 300  - Loss: 7.670573
I: 400  - Loss: 6.6367188
I: 500  - Loss: 6.404948
I: 600  - Loss: 6.651042
I: 700  - Loss: 6.5585938
I: 800  - Loss: 6.059896
I: 900  - Loss: 6.2981772
I: 1000  - Loss: 6.373698
I: 1100  - Loss: 6.7382812
I: 1200  - Loss: 6.4440107
I: 1300  - Loss: 6.359375
I: 1400  - Loss: 6.7057295
I: 1500  - Loss: 6.3919272
I: 1600  - Loss: 6.5898438
I: 1700  - Loss: 6.3138022
I: 1800  - Loss: 6.1054688
I: 1900  - Loss: 6.2148438
I: 2000  - Loss: 6.4570312
I: 2100  - Loss: 5.828125
I: 2200  - Loss: 6.186198
I: 2300  - Loss: 5.9179688
I: 2400  - Loss: 6.1484375
I: 2500  - Loss: 5.981771
I: 2600  - Loss: 5.8203125
I: 2700  - Loss: 6.3242188
I: 2800  - Loss: 6.4804688
I: 2900  - Loss: 6.0403647
I: 3000  - Loss: 6.6679688
I: 3100  - Loss: 6.545573
I: 3200  - Loss: 5.916667
I: 3300  - Loss: 6.4101562
I: 3400  - Loss: 6.4257812
I: 3500  - Loss: 5.6927085
I: 3600  - Loss: 5.6640625
I: 3700  - Loss: 6.260417

# Run the model

In [ ]:
# @title Autoregressive rollout (loop in python)

print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions

# Model running loop

N = eval_data.sizes['time'] - eval_steps - 4
# Open a zarr store to write predictions to disk
store = xr.backends.ZarrStore("../preds/predictions.zarr", "w")

for i in range(10):
    eval_batch = eval_data.isel(time=slice(i, i + eval_steps + 2))
    eval_batch = eval_batch.compute()

    eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    eval_batch, target_lead_times=slice("6h", f"{eval_steps*6}h"),
    **dataclasses.asdict(task_config))

    predictions = rollout.chunked_prediction(
        run_forward_jitted,
        rng=jax.random.PRNGKey(0),
        inputs=eval_inputs,
        targets_template=eval_targets * np.nan,
        forcings=eval_forcings)
    # Write the prediction dataset to the zarr store
    predictions.to_zarr(store, group=f"prediction_time_step_{prediction_time_step}", mode="a")

# Close the zarr store
store.close()

# Read the predictions back into memory and concatenate them along the 'pred_times' dimension
predictions = xr.open_zarr("../preds/predictions.zarr", concat_dim="prediction_time_step")
predictions

# Save the evaluation run to file


# Specify the path where you want to save the Zarr file
zarr_path = "Eval_preds/mse_64x32_2020.zarr"

# Save the dataset to the Zarr file
predictions.to_zarr(zarr_path, mode='w')

# Save the model to file

In [ ]:
import jax
import numpy as np
import jax.numpy as jnp
import os 

def flatten_dict(d, parent_key='', sep='//'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def save_model_params(d, file_path):
    flat_dict = flatten_dict(d)
    # Convert JAX arrays to NumPy for saving
    np_dict = {k: np.array(v) if isinstance(v, jnp.ndarray) else v for k, v in flat_dict.items()}
    np.savez(file_path, **np_dict)

params_path = os.path.join('../models', 'params_64x32_mae.npz')
save_model_params(params, params_path)

Now, our trained model is saved to a file ,which can be used to load and run again.